In [ ]:
# Vajalikud impordid
import random

# EstNLTK 
!pip install estnltk==1.7.2
from estnltk import Text
from estnltk.vabamorf.morf import synthesize

# YOLO
!pip install ultralytics -q
from ultralytics import YOLO

# Os 
import os

# CV
import cv2 as cv

# BeautifulSoup4
!pip install BeautifulSoup4
from bs4 import BeautifulSoup

In [ ]:
# YOLO (You Only Look Once) klassid piltidelt objektide tuvastamiseks.
# kommentaarina on jäetud algne klassi nimi.

klassid = {0: 'inimene', # person
 1: 'jalgratas', # bicycle
 2: 'auto', # car
 3: 'mootorratas', # motorcycle
 4: 'lennuk', # airplane
 5: 'buss', # bus
 6: 'rong', # train
 7: 'veoauto', # truck
 8: 'paat', # boat
 9: 'valgusfoor', # traffic light
 10: 'tuletõrjehüdrant', # fire hydrant
 11: 'stop märk', # stop sign
 12: 'parkimisautomaat', # parking meter
 13: 'pink', # bench
 14: 'lind', # bird
 15: 'kass', # cat
 16: 'koer', # dog
 17: 'hobune', # horse
 18: 'lammas', # sheep
 19: 'lehm', # cow
 20: 'elevant', # elephant 
 21: 'karu', # bear
 22: 'sebra', # zebra
 23: 'kaelkirjak', # giraffe
 24: 'seljakott', # backpack
 25: 'vihmavari', # umbrella
 26: 'käekott', # handbag
 27: 'lips', # tie
 28: 'kohver', # suitcase
 29: 'frisbee', # frisbee
 30: 'suusad', # skis
 31: 'lumelaud', # snowboard
 32: 'pall', # sports ball
 33: 'tuulelohe', # kite
 34: 'pesapallikurikas', # baseball bat
 35: 'pesapallikinnas', # baseball glove
 36: 'rula', # skateboard
 37: 'surfilaud', # surfboard
 38: 'reket', # tennis racket
 39: 'pudel', # bottle
 40: 'klaas', # wine glass
 41: 'kruus', # cup
 42: 'kahvel', # fork
 43: 'nuga', # knife
 44: 'lusikas', # spoon
 45: 'kauss', # bowl
 46: 'banaan', # banan
 47: 'õun', # apple 
 48: 'võileib', # sandwich
 49: 'apelsin', # orange
 50: 'brokoli', # broccoli
 51: 'porgand', # carrot
 52: 'hot dog', # hot dog
 53: 'pitsa', # pizza
 54: 'sõõrik', # donut
 55: 'tort', # cake
 56: 'tool', # chair
 57: 'diivan', # couch
 58: 'potilill', # potted plant
 59: 'voodi', # bed
 60: 'laud', # dining table
 61: 'tualett', # toilet
 62: 'televiisor', # tv
 63: 'arvuti', # laptop
 64: 'arvutihiir', # mouse
 65: 'pult', # remote
 66: 'klaviatuur', # keyboard
 67: 'mobiiltelefon', # cell phone
 68: 'mikrolaineahi', # microwave
 69: 'ahi', # oven
 70: 'röster', # toaster
 71: 'kraanikauss', # sink
 72: 'külmkapp', # refrigerator
 73: 'raamat', # book
 74: 'kell', # clock
 75: 'vaas', # vase
 76: 'käärid', # scissors 
 77: 'mänguasi', # teddy bear
 78: 'föön', # hair drier
 79: 'hambahari'} # toothbrush

In [ ]:
pildi_kaust = "Images" # Clipart1k piltide kaust

# Funktsioon valitud pildilt objektide tuvastamiseks
def leia_klassid(tuvastatav_pilt):
    tagastus = None
    koopia = None 
    
    # Kasutatud abi objektide tuvastamisel YOLO-ga https://docs.ultralytics.com/modes/predict/#inference-arguments
    mudel = YOLO("yolov8n.pt")
    ennustus = mudel.predict("pildid/" + tuvastatav_pilt)

    # Leiame kõik objektid pildilt
    tensor = ennustus[0].boxes.cls 
    
    # Kogume kõik objektid hulka, selleks et vältida kordusi 
    valikud = set()
    for i in range(len(tensor)):
        valikud.add(tensor[i].item())
    if len(valikud) >= 4:
        tagastus = list(valikud)[:4]
        koopia = list(valikud)[:4]
        
    # Juhul kui pilt on sobiv, siis genereerime sellele vastava pildijärjendi
    if tagastus is not None:    
        pilt = cv.imread("pildid/" + tuvastatav_pilt)
        
        # Järgnevalt teeme pildist koopiad peidetud objektidega
        indeks = 4
        värv = [215, 235, 250] # Objektide katmiseks kasutatav värv
        for i in range(len(ennustus[0].boxes.cls)):
            if not ennustus[0].boxes.cls[i] in koopia:
                continue
            coords = [int(x) for x in ennustus[0].boxes.xyxy[i]] # Leiame objekti koordinaadid
            for j in range(coords[1], coords[3]): # Kirjutame leitud objekti beežiga üle
                for k in range(coords[0], coords[2]):
                    for l in range(3):
                        pilt[j][k][l] = värv[l] 
                        
            # Lisame peidetud objektiga pildi "pildid" kausta, nimeks anname algsepildinimi-mitmendagateguon.jpg
            cv.imwrite("pildid/" + tuvastatav_pilt.split(".")[0] + "-" + str(indeks) + ".jpg", pilt)
            indeks -= 1
            koopia.remove(ennustus[0].boxes.cls[i])
            
    # Kui pilt sobib, siis tagastame pildilt leitud objektid
    if tagastus is not None:
        return [klassid[v] for v in tagastus]
    return None

# Järgneva funktsiooniga leitakse kõik sobivad pildid. 
# Sobivad pildid on need, millel on vähemalt neli YOLO poolt tuvastatud klassi.
def eeltöötlus_piltidele():
    sobivad_pildid = []
    for faili_nimi in os.listdir("pildid"):
        fail = os.path.join("pildid", faili_nimi)
        klassid = leia_klassid(faili_nimi)
        if klassid is not None:
            sobivad_pildid.append(faili_nimi)

In [ ]:
# Funktsioon kirjutab igale pildile tekstifaili, kus sees on pildil olevad tuvastatud objektid
def pildi_tekstifail():
    pildi_kaust = "pildid" 
    for pilt in os.listdir(pildi_kaust):
        valikud = leia_klassid(pilt)
        if valikud is None:
            continue
        with open("pildid/" + pilt.split(".")[0] + ".txt", "w", encoding="utf-8") as f:
            for objekt in valikud:
                f.write(objekt)
                f.write("\n")

In [ ]:
# Tekstide eeltöötlus
# Funktsioon loeb sisse ilukirjandus tekstid, eemaldab XML-märgendid ja kontrollib kas lause sisaldab YOLO klassi
def töötle_failid():
    failide_kaust = "Eesti_ilukirjandus"

    töödeldud = {} # teeme korpuse, kus on eraldatud laused
    for faili_nimi in os.listdir(failide_kaust):
        fail = os.path.join(failide_kaust, faili_nimi)
        try:
            with open(fail, 'r',  encoding='utf-8') as f:
                contents = f.read()
                
                # kasutatud abi XML märgendite eemaldamiseks https://oxylabs.io/blog/beautiful-soup-parsing-tutorial 
                soup = BeautifulSoup(contents, "html.parser") # eemaldame märgendid tekstist
                
                for tag in soup.find_all('s'): # <s> tagide sisu (tekst)
                    tekst = Text(tag.text).tag_layer()
                    
                    # Kasutatud abi EstNLTK https://github.com/estnltk/estnltk/blob/main/tutorials/nlp_pipeline/B_morphology/01_morphological_analysis.ipynb
                    for lemma in tekst.morph_analysis.lemma: # lauses esinevad lemmad
                        if lemma in klassid.values(): # kui sõna on klass siis jätame lause meelde
                            if töödeldud.has_key(klass):
                                töödeldud[klass].append(tag.text)
                            else:
                                töödeldud[klass] = [tag.text]

        except FileNotFoundError:
            print(f"Faili '{fail_tekstina}' ei leitud.")
    return töödeldud

In [ ]:
# Funktsioon leiab lauses sõnade liigi, leiame sõna, mis on sihitis
def leia_lause_osad(lause):
    tekst = Text(lause).tag_layer()
    tekst.tag_layer('maltparser_syntax') # Formaaditeisendus ja süntaksianalüüs
    sonad = tekst.maltparser_syntax["text"]
    a = tekst.maltparser_syntax["deprel"]

    for i, s in enumerate(sonad):
        if a[i] == "obj": # "obj" on sihitis
            return s
    return None

In [ ]:
# Funktsioon sünteesib leitud sihitisele kaks varianti juurde
def tee3varianti(s):
    tagastus = [s]

    lemma = Text(s).tag_layer().morph_analysis.lemma[0][0]

    a = ["sg", "pl"] # ainsus ja mitmus
    b = ["ab", "abl", "ad", "all", "el", "es", "ill", "in", "kom", "ter", "tr"] # käänded
    random.shuffle(a)
    random.shuffle(b)
    variandid = []
    
    # Valime kas ainsuse või mitmuse
    for valikA in a:
        # Valime iga käände
        for valikB in b:
            if len(tagastus) + len(variandid) >= 3:
                # Juhul kui kokku on vähemalt 3 varianti, siis tagastame need
                return tagastus + random.sample(variandid, 3 - len(tagastus))
            # Muul juhul sünteesime uue variandi
            tulemus = synthesize(lemma, valikA + " " + valikB)
            # Juhul kui seda ei ole või see on algse sihitisega sama, siis jätame vahele
            if not tulemus or tulemus[0] == s or tulemus[0] == tagastus[-1]:
                continue
            variandid.append(tulemus[0])
    return None

In [ ]:
# Lausete kirjavahemärkide vahel on tühikud
# Funktsioon lausete korrastamiseks
def lause_korrastamine(lause):
    korrastatud_lause = ""

    for i in lause.split(" "):
        if i == '-':
            korrastatud_lause += " - "
        elif i == ","  or i == ":" or i == "!" or i == "?" or i == "." or i == ')' or i == ';':
            korrastatud_lause += i
        else:
            tühik = True
            if len(korrastatud_lause) != 0:
                if korrastatud_lause[-1] == ' ' or korrastatud_lause[-1] == '(':
                    tühik = False
            if tühik:
                korrastatud_lause += " "
            korrastatud_lause += i
    korrastatud_lause = korrastatud_lause[1:]
    return korrastatud_lause

In [ ]:
# Funktsioon korpusest suvalise lause leidmiseks pildi järgi
# Laused: dict {klass: lausete list}, klass: sõne
def lause_leidmine(laused, klass):
    lause = random.choice(laused[klassid[int(random.choice(list(klass)))]])
    return lause

In [ ]:
# Funktsioon vastavate klasside lausete tekstifaili kirjutamine
def kirjuta_töödeldud(töödeldud):
    for v in töödeldud: # valime klassi
        fail = open(os.path.join("valmis", klassid[v] + ".txt"), "w", encoding="utf-8")
        for lause in töödeldud[v]:
            fail.write(lause)
            fail.write("\n")
        fail.close()

In [ ]:
# teeme uue kausta töödeldud klassidega
def loe_töödeldud():
    töödeldud = {}
    for fail in os.listdir("valmis"):
        nimi = fail.split(".")[0]
        töödeldud[nimi] = []
        f = open(os.path.join("valmis", fail), "r", encoding="utf-8")
        rida = f.readline().strip()
        while rida:
            töödeldud[nimi].append(rida)
            rida = f.readline().strip()
        f.close()
    return töödeldud

In [ ]:
# Peafunktsioon
# Funktsioon ühe mängu korra imiteerimiseks (nüüd on lõpp versioonis kogu eeltöötlus tehtud ehk andmed loetakse sisse)
def peafunktsioon():
    laused = loe_töödeldud()
    sihitis = None
    while sihitis == None:
        klass = None
        while klass == None:
            pilt = vali_pilt()
            klass = leia_klassid(pilt)
            if klass != None:
                img = cv.imread(pildi_kaust + "/" + pilt)
                cv.imshow('Pealkiri', img)
                cv.waitKey(0)
                cv.destroyAllWindows()
        lause = lause_leidmine(laused, klass)
        korrastatud = lause_korrastamine(lause)
        sihitis = leia_lause_osad(korrastatud)
        
    variandid = tee3varianti(sihitis)
    print(korrastatud.replace(sihitis, "_ _ _"))
    print(sihitis)
    print(variandid)

In [ ]:
# Filtreeritakse välja laused, milles ei ole sihitist
def töötle_laused(laused):
    uus = {}
    for key in laused.keys():
        for lause in laused[key]:
            tulemus = leia_lause_osad(lause)
            if tulemus is None:
                continue
            if key in uus:
                uus[key].append(lause)
            else:
                uus[key] = [lause]    
    return uus

In [ ]:
# Lausete kokku lugemiseks
lauseid = 0

for faili_nimi in os.listdir("valmis"):
    l = leia_laused_sihitistega(faili_nimi)
    lauseid += len(l)
    kirjutaLaused(l, faili_nimi)
    
print(lauseid)

In [ ]:
# Abimeetod lausete kirjutamiseks
def kirjutaLaused(laused, faili_nimi):
    with open("sihitistega/" + faili_nimi, "w", encoding="utf-8") as f:
        for lause in laused:
            f.write(lause)
            f.write("\n")

In [ ]:
# Laused olid failis kirjas formaadis lause.|sihitis ning sihitis oli vaja ära käänata
for faili_nimi in os.listdir("sihitistega"):
    laused = None
    with open("sihitistega/" + faili_nimi, 'r', encoding = 'utf-8') as f:
        laused = f.readlines()
    
    laused = sihitiseKäänamine(laused)
    
    with open('sihitistega/' + faili_nimi, 'w', encoding = 'utf-8') as f:
        for lause in laused:
            f.write(lause)
            f.write('\n')
    

In [ ]:
# Funktsioon sihitise variantide genereerimiseks
def sihitiseKäänamine(laused):
    sihitistega = []
    for lause in laused:
        osad = lause.split("|")
        terviklause = osad[0]    
        for sihitis in osad[1:]:
            õige = sihitis.split(";")[0]
            variandid = tee3varianti(õige)
            terviklause += "|" + ";".join(variandid)
        sihitistega.append(terviklause)
    return sihitistega

In [ ]:
# Funktsioon sihitiste leidmiseks ja lisamiseks lause lõppu
# Seda funktsiooni kasutati, siis kui andmed olid kirjutatud teises formaadis
def leia_laused_sihitistega(failitee):
    leitud = []
    failide_kaust = "valmis"
    fail = os.path.join(failide_kaust, failitee)
    try:
        with open(fail, 'r',  encoding='utf-8') as f:
            sisu= " ".join(f.readlines())
            sisu = sisu.replace("\n", " ")
            tekst = Text(sisu).tag_layer()
            tekst = tekst.tag_layer('maltparser_syntax') # Formaaditeisendus ja süntaksianalüüs
            laused = tekst.sentences
            sonad = tekst.maltparser_syntax["text"]
            a = tekst.maltparser_syntax["deprel"]
            indeks = 0
            for lause in laused:
                sihitised = []
                for i in range(indeks, indeks + len(lause)):
                    if a[i] == "obj":
                        sihitised.append(sonad[i])
                if len(sihitised) > 0:
                    # Lisame sihitised lause lõppu eraldades lausest püstkriipsuga
                    leitud.append(" ".join(lause.text) + "|" + "|".join(sihitised))
                indeks += len(lause)
    except FileNotFoundError:
        print(f"Faili '{fail}' ei leitud.")
    return leitud

In [ ]:
# Funktsioon sihitiste variantide genereerimiseks
def sihitise_variandid(failitee):
    leitud = []
    failide_kaust = "sihitistega"
    fail = os.path.join(failide_kaust, failitee)
    try:
        with open(fail, 'r',  encoding='utf-8') as f:
            sisu = f.readlines() # lausete list
            for lause in sisu:
                lause = lause.replace('\n', "")
                osad = lause.split("|") # Lause.|sihitis,variant1,variant2
                uus_lause = [lause_korrastamine(osad[0])] # Lausel tehakse kirjavahemärgistus korda
                for sihitis in osad[1:]:
                    variandid = tee3varianti(sihitis) # Genereeritakse sihitise variandid
                    if variandid is None:
                        continue
                    uus_lause.append(";".join(variandid)) # Sihitised pannakse ";" kokku
                if len(uus_lause) > 1:
                    leitud.append("|".join(uus_lause)) # Lause pannakse sihitistega kokku püstkriipsuga
    except FileNotFoundError:
        print(f"Faili '{fail}' ei leitud.")
    return leitud

In [ ]:
# Lausele sihitiste variantide genereerimine ja kirjutamine
for faili_nimi in os.listdir("sihitistega"):
    l = sihitise_variandid(faili_nimi)
    kirjutaLaused(l, faili_nimi)